In [ ]:
import gc
import glob
import random
from pathlib import Path
from tempfile import gettempdir

import matplotlib as mpl
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
import scipy as sp
import scipy.constants
from IPython.core.display import HTML, display
from matplotlib import cm, ticker
from numpy import ma

In [ ]:
COLORS = mpl.rcParams["axes.prop_cycle"].by_key()["color"]

# Toggle on/off interactive notebooks
if False:
    %matplotlib notebook
    FIGSIZE = 6
else:
    %matplotlib inline
    FIGSIZE = 12

mpl.rcParams["figure.figsize"] = [FIGSIZE, FIGSIZE / sp.constants.golden]

In [ ]:
def random_drop(df, count=100_000, inplace=False):
    ratio = count / df.index.size
    selector = list(filter(lambda _: random.random() > ratio, df.index))
    return df.drop(selector, inplace=inplace)

In [ ]:
OUTPUT_DIR = Path(gettempdir()) / "boltzmann_solver" / "leptogenesis"
# OUTPUT_DIR = (
#     Path("/media/ssh/uni-josh") / "tmp" / "josh" / "boltzmann_solver" / "leptogenesis"
# )
# OUTPUT_DIR = Path("../output/") / "leptogenesis"
data = dict()

# Masses and Widths

In [ ]:
data["mass"] = pd.read_csv(OUTPUT_DIR / "mass.csv").sort_values(by="beta")
data["width"] = pd.read_csv(OUTPUT_DIR / "width.csv").sort_values(by="beta")

data["mass beta"] = data["mass"].copy()
data["mass beta"].iloc[:, 1:] = (
    data["mass"].iloc[:, 1:].multiply(data["mass"].iloc[:, 0], axis="index")
)

data["width / mass"] = data["width"].copy()
data["width / mass"].iloc[:, 1:] = (
    data["width"].iloc[:, 1:] / data["mass"].iloc[:, 1:]# .multiply(data["mass"].iloc[:, 0], axis="index")
)

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)

data["mass beta"].plot(x="beta", y=["A", "W", "H", "L3", "e3", "Q3", "u3", "d3"], ax=axs[0])
data["mass"].plot(x="beta", y=data["mass"].columns[-3:], ax=axs[1])

for ax in axs:
    ax.set_xlabel(r"Inverse Temperature [GeV$^{-1}$]")
    ax.set_xscale("log")
    ax.set_ylabel("Mass / Temperature")
    ax.set_yscale("log")
axs[-1].set_ylabel("Mass [GeV]")

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)

data["width / mass"].plot(x="beta", y=["H"], ax=axs[0])
data["width / mass"].plot(x="beta", y=["N1", "N2", "N3"], ax=axs[1])

for ax in axs:
    ax.set_xlabel(r"Inverse Temperature [GeV$^{-1}$]")
    ax.set_xscale("log")
    ax.set_ylabel("Mass / Temperature")
    ax.set_yscale("log")
axs[-1].set_ylabel("Mass [GeV]")

pyplot.show()

# Decay Only

## 1 Generation

In [ ]:
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_only" / "1gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
# data["n"].plot(x="beta", y=["ΔN1", "ΔN2", "ΔN3"])
# data["n"].abs().plot(x="beta", y=data["n"].columns[2:3], ax=axs[0])
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-30)
# dbl = axs[0].twinx()
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    legend=False,
)
data["n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["N1", "N2", "N3"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(N1)", "(N2)", "(N3)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    legend=False,
)
# axs[1].set_yscale("log")
axs[1].set_ylim(bottom=0, top=2)

# ## Change in Number Density
# axs[2].set_title("Change in Number Densities")
# data["n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
# )
# data["-n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
#     linestyle="dashed",
# )
# axs[2].set_yscale("log")
# axs[2].set_ylim(bottom=1e-40)

pyplot.show()

## 3 Generations

In [ ]:
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_only" / "3gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
# data["n"].plot(x="beta", y=["ΔN1", "ΔN2", "ΔN3"])
# data["n"].abs().plot(x="beta", y=data["n"].columns[2:3], ax=axs[0])
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-30)
# dbl = axs[0].twinx()
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    legend=False,
)
data["n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["N1", "N2", "N3"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(N1)", "(N2)", "(N3)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    legend=False,
)
# axs[1].set_yscale("log")
axs[1].set_ylim(bottom=0, top=2)

# ## Change in Number Density
# axs[2].set_title("Change in Number Densities")
# data["n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
# )
# data["-n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
#     linestyle="dashed",
# )
# axs[2].set_yscale("log")
# axs[2].set_ylim(bottom=1e-40)

pyplot.show()

# Decay + $\Delta L = 2$

## 1 Generation

In [ ]:
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_washout" / "1gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
# data["n"].plot(x="beta", y=["ΔN1", "ΔN2", "ΔN3"])
# data["n"].abs().plot(x="beta", y=data["n"].columns[2:3], ax=axs[0])
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-30)
# dbl = axs[0].twinx()
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    legend=False,
)
data["n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["N1", "N2", "N3"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(N1)", "(N2)", "(N3)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    legend=False,
)
# axs[1].set_yscale("log")
axs[1].set_ylim(bottom=0, top=2)

# ## Change in Number Density
# axs[2].set_title("Change in Number Densities")
# data["n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
# )
# data["-n"].plot(
#     x="beta",
#     y=data["n"].columns[7::3],
#     ax=axs[2],
#     color=COLORS,
#     linestyle="dashed",
# )
# axs[2].set_yscale("log")
# axs[2].set_ylim(bottom=1e-40)

pyplot.show()

# Gammas

In [ ]:
files = sorted(glob.glob(str(OUTPUT_DIR / "gamma" / "*.csv")))

data["gamma"] = dict()
for file in files:
    number = int(file.split('/')[-1].split('.csv')[0])
    data["gamma"][number] = pd.read_csv(file)

In [ ]:
for key in data["gamma"].keys():
    data["gamma"][key].plot(x="beta", loglog=True)

# Scan

In [ ]:
data["scan"] = pd.read_csv(OUTPUT_DIR / "scan.csv").sort_values(by=["y", "m"])

gc.collect()

In [ ]:
x = data["scan"]["y"]
y = data["scan"]["m"]
nx = x.unique().size
ny = y.unique().size
x = x.values.reshape(nx, ny)
y = y.values.reshape(nx, ny)
z = data["scan"]["B-L"].abs().values.reshape(nx, ny)
z = ma.masked_where(np.isnan(z), z)

fig, ax = pyplot.subplots()

ax.set_xlabel("$y_v$")
ax.set_xscale("log")
ax.set_ylabel("$N_1$ mass [GeV]")
ax.set_yscale("log")
cs = ax.contourf(x, y, z, locator=ticker.LogLocator())
# cs = ax.contourf(x, y, z)
# ax.plot(x, y, "ko", ms=3)
cbar = fig.colorbar(cs, ticks=ticker.LogLocator())
# cbar = fig.colorbar(cs)
cbar.set_label("B-L")

pyplot.show()